In [1]:
import pandas as pd
import time
import random

from bs4 import BeautifulSoup as bs
import requests
import numpy as np

from IPython.display import display
from datetime import datetime

from dotenv import load_dotenv
import os
load_dotenv()

frescos = {
    # "Frutas": "https://www.alcampo.es/compra-online/frescos/frutas/c/W1701?q=%3Arelevance&page=",
    # "Carne": "https://www.alcampo.es/compra-online/frescos/carne/c/W13?q=%3Arelevance&page=",
    # "Panadería":"https://www.alcampo.es/compra-online/frescos/panaderia/c/W1281?q=%3Arelevance&page=",
    # "Verduras y hortalizas":"https://www.alcampo.es/compra-online/frescos/verduras-y-hortalizas/c/W1702?q=%3Arelevance&page=",
    # "Charcutería":"https://www.alcampo.es/compra-online/frescos/charcuteria/c/W15?q=%3Arelevance&page=",
    # "Pastelería":"https://www.alcampo.es/compra-online/frescos/pasteleria/c/W1282?q=%3Arelevance&page=",
    # "Sushi": "https://www.alcampo.es/compra-online/frescos/sushi/c/W140502?q=%3Arelevance&page=",
    "Quesos": "https://www.alcampo.es/compra-online/frescos/quesos/c/WQuesos?q=%3Arelevance&page=",
    "Pescados, mariscos y moluscos": "https://www.alcampo.es/compra-online/frescos/pescados-mariscos-y-moluscos/c/W14?q=%3Arelevance&page=",
    "Ahumados, sucedáneos gulas, anchoas": "https://www.alcampo.es/compra-online/frescos/ahumados-sucedaneos-gulas-anchoas/c/W184?q=%3Arelevance&page=",
    "Fresco al Corte": "https://www.alcampo.es/compra-online/frescos/fresco-al-corte/c/WFrescoalcorte?q=%3Arelevance&page="
}



dict_alcampo = {"Category": [], "Name":[],"Price":[],"Stock":[], "Image":[]}

for i in frescos.items():

    categoria = i[0]
    url_to_scrape = i[1]
    contador = 0
    product_not_found = 10

    while product_not_found != "0":
        
        url = url_to_scrape+ f"{contador}"
        response = requests.get(url='https://proxy.scrapeops.io/v1/',params={'api_key': os.environ["SCARPE_OPS_API_KEY"],'url': url, },)
        html = response.content
        soup = bs(html,"lxml")
     
        print(categoria,url)
        try:
            product_not_found = soup.find("div",class_="totalResults").get_text(strip=True, separator='').split(" ")[0]
        except:
            print("Got an attribute error on:",url)

        if product_not_found =="0":
            print("No hay más ventanas")
            break
        
        else:
            products = soup.find_all("div",class_="productGridItemContainer")
            prices = soup.find_all("div",class_="priceContainer")
            stock_available = soup.find_all("div",class_="cart add")

            for product in products:

                dict_alcampo["Name"].append(product.find("span").get_text(strip=True))
                dict_alcampo["Image"].append(product.find("img")["src"])

                dict_alcampo["Category"].append(categoria)

            for price in prices:
                try:
                    dict_alcampo["Price"].append(price.find_all("span")[2].get_text(strip=True))
                except:
                    print("List index out of range")
                    dict_alcampo["Price"].append("NaN")

            for stock in stock_available:
                
                stock_yes_no = stock.find_all("button")[2].get_text(strip=True).split(" "[0])
                
                if stock_yes_no[0]=="Sin":
                    dict_alcampo["Stock"].append("No")
                else:
                    dict_alcampo["Stock"].append("Yes")
                              
            print( "Category:", len(dict_alcampo["Category"]), "Name:", len(dict_alcampo["Name"]) ,"Image:",len(dict_alcampo["Image"]),"Price:", len(dict_alcampo["Price"]), "Stock",len(dict_alcampo["Stock"])    )
            
        contador+=1

df_alcampo = pd.DataFrame(dict_alcampo)

df_alcampo








Quesos https://www.alcampo.es/compra-online/frescos/quesos/c/WQuesos?q=%3Arelevance&page=0
Got an attribute error on: https://www.alcampo.es/compra-online/frescos/quesos/c/WQuesos?q=%3Arelevance&page=0
Category: 0 Name: 0 Image: 0 Price: 0 Stock 0
Quesos https://www.alcampo.es/compra-online/frescos/quesos/c/WQuesos?q=%3Arelevance&page=1
Got an attribute error on: https://www.alcampo.es/compra-online/frescos/quesos/c/WQuesos?q=%3Arelevance&page=1
Category: 0 Name: 0 Image: 0 Price: 0 Stock 0
Quesos https://www.alcampo.es/compra-online/frescos/quesos/c/WQuesos?q=%3Arelevance&page=2
Category: 48 Name: 48 Image: 48 Price: 48 Stock 48
Quesos https://www.alcampo.es/compra-online/frescos/quesos/c/WQuesos?q=%3Arelevance&page=3
Category: 96 Name: 96 Image: 96 Price: 96 Stock 96
Quesos https://www.alcampo.es/compra-online/frescos/quesos/c/WQuesos?q=%3Arelevance&page=4
Category: 144 Name: 144 Image: 144 Price: 144 Stock 144
Quesos https://www.alcampo.es/compra-online/frescos/quesos/c/WQuesos?q=%3

KeyboardInterrupt: 